In [34]:
# 2️⃣ OpenAI API Key ayarla
# -------------------------------
import os
os.environ["OPENAI_API_KEY"] = ""  # kendi key'ini buraya yapıştır

In [35]:
# 3️⃣ Kütüphaneler
# -------------------------------
import fitz  # PyMuPDF
import pandas as pd
from tqdm import tqdm
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
import csv

In [36]:
# 4️⃣ Dokümanlardan Metin Çekme
# -------------------------------
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def extract_text_from_csv(csv_path):
    df = pd.read_csv(csv_path)
    texts = []
    for _, row in tqdm(df.iterrows(), total=df.shape[0], desc="CSV Satırlarını İşliyor"):
        row_text = ' | '.join(row.astype(str))
        texts.append(row_text)
    return '\n'.join(texts)

In [37]:
# Dosya yolları
pdf_text = extract_text_from_pdf("/content/Document 1.pdf")
csv_text = extract_text_from_csv("product_specs.csv")

print(f"PDF metin uzunluğu: {len(pdf_text)} karakter")
print(f"CSV metin uzunluğu: {len(csv_text)} karakter")

CSV Satırlarını İşliyor: 100%|██████████| 3000/3000 [00:00<00:00, 5439.10it/s]

PDF metin uzunluğu: 2980 karakter
CSV metin uzunluğu: 146002 karakter


In [38]:
# 5️⃣ Text Splitter (Token limitini önlemek için)
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# CSV'yi bloklara ayır
chunk_size_lines = 500
csv_lines = csv_text.split('\n')
csv_chunks = [csv_lines[i:i+chunk_size_lines] for i in range(0, len(csv_lines), chunk_size_lines)]

docs = []
for chunk in csv_chunks:
    chunk_text = "\n".join(chunk)
    docs += text_splitter.create_documents([chunk_text])

# PDF metni için de doküman oluştur
docs += text_splitter.create_documents([pdf_text])

In [39]:
# 6️⃣ Embedding oluştur ve Chroma vector store
# -------------------------------
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(docs, embeddings, persist_directory="chroma_db")

# RAG Retrieval QA zinciri
retriever = vectordb.as_retriever(search_kwargs={"k":3})
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"),
    retriever=retriever
)


In [ ]:
import re

# 9️⃣ Sorgu Döngüsü
while True:
    query = input("Soru: ")
    if query.lower() in ['exit', 'quit']:
        print("Programdan çıkılıyor.")
        break

    # CSV ürün bazlı filtreleme
    if "product" in query.lower() or "ürün" in query.lower():
        product_ids = re.findall(r'#?(\d+)', query)
        docs_temp = []

        for pid in product_ids:
            filtered_rows = df[df["Product"].astype(str).str.contains(f"#{pid}")]
            if not filtered_rows.empty:
                csv_text_filtered = "\n".join(filtered_rows.astype(str).apply(lambda x: " | ".join(x), axis=1))
                docs_temp += text_splitter.create_documents([csv_text_filtered])

        docs_temp += text_splitter.create_documents([pdf_text])

        vectordb_temp = Chroma.from_documents(docs_temp, embeddings)
        retriever_temp = vectordb_temp.as_retriever(search_kwargs={"k":3})
        qa_chain_temp = RetrievalQA.from_chain_type(
            llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"),
            retriever=retriever_temp
        )
        answer = qa_chain_temp.invoke({"query": query})["result"]

    else:
        answer = qa_chain.invoke({"query": query})["result"]

    print("\nCevap:", answer, "\n")

    # Loglama
    with open(log_file, 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([query, answer])

Soru: En ağır 5 ürünü (Weight (kg)) sırala ve listeler misin?

Cevap: En ağır 5 ürünü (Weight (kg)) sıralayarak listeliyorum:
1. Product D #588 | 24.97 kg
2. Product A #557 | 24.46 kg
3. Product X #523 | 24.55 kg
4. Product B #547 | 24.1 kg
5. Product B #554 | 23.53 kg 

Soru: Weight (kg) sütununa göre en ağır ürünler hangileri?

Cevap: En ağır ürünleri belirlemek için ürünlerin ağırlıklarını kontrol etmek gerekmektedir. En ağır ürünlerin listesi aşağıdaki gibidir:

1. Product D #588 | 24.97 kg
2. Product A #557 | 24.46 kg
3. Product X #573 | 24.55 kg
4. Product B #547 | 24.1 kg
5. Product B #555 | 21.73 kg 

Soru: En fazla ürün hangi Material türünde?

Cevap: En fazla ürün Titanium malzeme türündedir. Toplamda 11 adet ürün Titanium malzemesine sahiptir. 

Soru: Quantity Available en yüksek olan ürünler hangileri?

Cevap: En yüksek miktarda stokta bulunan ürünler aşağıdaki gibidir:

- Ürün A #53 | 297 adet
- Ürün C #541 | 299 adet
- Ürün D #503 | 286 adet
- Ürün A #557 | 283 adet
- Ürü